In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
import xgboost as xgb

class RBM:
    def __init__(self, visible_dim, hidden_dim, learning_rate=0.01):
        self.visible_dim = visible_dim
        self.hidden_dim = hidden_dim
        self.learning_rate = learning_rate

        # Initialize weights and biases
        self.weights = tf.Variable(tf.random.normal([visible_dim, hidden_dim], mean=0.0, stddev=0.01), dtype=tf.float32)
        self.visible_bias = tf.Variable(tf.zeros([visible_dim]), dtype=tf.float32)
        self.hidden_bias = tf.Variable(tf.zeros([hidden_dim]), dtype=tf.float32)

    # Gibbs sampling steps
    def sample_hidden(self, visible):
        hidden_probs = tf.nn.sigmoid(tf.matmul(visible, self.weights) + self.hidden_bias)
        hidden_states = tf.nn.relu(tf.sign(hidden_probs - tf.random.uniform(tf.shape(hidden_probs))))
        return hidden_states, hidden_probs

    def sample_visible(self, hidden):
        visible_probs = tf.nn.sigmoid(tf.matmul(hidden, tf.transpose(self.weights)) + self.visible_bias)
        visible_states = tf.nn.relu(tf.sign(visible_probs - tf.random.uniform(tf.shape(visible_probs))))
        return visible_states, visible_probs

    # Contrastive divergence
    def contrastive_divergence(self, input_data, k=1):
        visible_states = input_data

        # Positive phase
        hidden_states_pos, hidden_probs_pos = self.sample_hidden(visible_states)

        # Gibbs sampling for k steps
        hidden_states = hidden_states_pos
        for _ in range(k):
            visible_states, _ = self.sample_visible(hidden_states)
            hidden_states, hidden_probs_neg = self.sample_hidden(visible_states)

        # Update weights and biases
        positive_grad = tf.matmul(tf.transpose(input_data), hidden_probs_pos)
        negative_grad = tf.matmul(tf.transpose(visible_states), hidden_probs_neg)

        self.weights.assign_add(self.learning_rate * (positive_grad - negative_grad) / tf.cast(tf.shape(input_data)[0], tf.float32))
        self.visible_bias.assign_add(self.learning_rate * tf.reduce_mean(input_data - visible_states, axis=0))
        self.hidden_bias.assign_add(self.learning_rate * tf.reduce_mean(hidden_probs_pos - hidden_probs_neg, axis=0))

    # Training function
    def train(self, data, epochs=10, batch_size=64, k=1):
        dataset = tf.data.Dataset.from_tensor_slices(data).batch(batch_size)
        for epoch in range(epochs):
            for batch in dataset:
                self.contrastive_divergence(batch, k)
            print(f'Epoch {epoch+1} completed.')

    # Transform visible data to hidden representation
    def transform(self, data):
        _, hidden_probs = self.sample_hidden(data)
        return hidden_probs.numpy()

In [26]:
df = pd.read_csv('../data/files/mel_spectograms_flat.csv')

In [27]:
from sklearn.model_selection import train_test_split

def clean_labels(s):
    return s.split('.')[0]

df[df.columns[1]] = df[df.columns[1]].apply(lambda x: clean_labels(x))
df.head(10)

labelMap = {
    "blues": 0,
    "classical": 1,
    "country": 2,
    "disco": 3,
    "hiphop": 4,
    "jazz": 5,
    "metal": 6,
    "pop": 7,
    "reggae": 8,
    "rock": 9
}

X = df.drop([df.columns[0], df.columns[1]], axis=1)
y = list(map(lambda x: labelMap[x], df[df.columns[1]]))

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1122)

In [28]:
labels = labelMap.values()
clf = xgb.XGBClassifier(
    objective='multi:softmax',
    n_estimators=500,
    learning_rate=0.05,
    max_depth=3,
    random_state=1122,
    eval_metric='mlogloss',
    early_stopping_rounds=5,
    n_jobs=-1,
)

clf.fit(X_train, y_train, verbose=1, eval_set=[(X_train, y_train), (X_test, y_test)])

predictions = clf.predict(X_test)
print('Accuracy:', accuracy_score(y_test, predictions))

import matplotlib.pyplot as plt
import seaborn as sns

def plot_confusion_matrix(y_true, y_pred, labels):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10,7))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=labels, yticklabels=labels, cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Multiclass Confusion Matrix')
    plt.show()

# Call function:
plot_confusion_matrix(y_test, predictions, labels)

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:1: object

In [12]:
print(classification_report(y_true=y_test, y_pred=predictions))

              precision    recall  f1-score   support

           0       0.35      0.25      0.29        24
           1       0.68      0.83      0.75        18
           2       0.36      0.30      0.33        27
           3       0.61      0.58      0.60        24
           4       0.54      0.65      0.59        20
           5       0.55      0.61      0.58        28
           6       0.77      0.83      0.80        29
           7       0.62      0.91      0.74        23
           8       0.67      0.35      0.46        34
           9       0.39      0.48      0.43        23

    accuracy                           0.56       250
   macro avg       0.55      0.58      0.56       250
weighted avg       0.56      0.56      0.55       250



In [29]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

input_shape = X_train.shape[1:]  # (128, time_frames, 1)
num_classes = len(labels)

model = Sequential([
    Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    BatchNormalization(),
    MaxPooling2D((2,2)),
    
    Conv2D(64, kernel_size=(3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2,2)),
    
    Conv2D(128, kernel_size=(3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

C:\Users\awast\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Input 0 of layer "conv2d_9" is incompatible with the layer: expected min_ndim=4, found ndim=2. Full shape received: (None, 1)

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=32,
    validation_split=0.2
)

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Accuracy: {test_acc:.2%}")

y_pred = model.predict(X_test).argmax(axis=1)

print(classification_report(y_test, y_pred, target_names=labels))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=labels, yticklabels=labels, cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()